In [5]:
# 在 Jupyter 中模拟 stdin 调试
import sys, io, math, heapq

# <<< 在这里编辑你的测试数据，多行就是逐行输入 >>>
sample_input = """A100-人民路15号-30
A101-中山公园南门-15
A102-科技大厦B座-25
A103-绿城花园8栋-20
end
"""
sys.stdin = io.StringIO(sample_input)

In [2]:
import sys
import heapq

heap = [(0, 1), (0, 2), (0, 3)]  # (空闲时间, 骑手id)
task = []

for line in sys.stdin:
    line = line.strip()
    if line == "end":
        break

    id_order, rest = line.split("-", 1)
    address, time = rest.rsplit("-", 1)
    time = int(time)

    free_time, id_rider = heapq.heappop(heap)  # 取出最早空闲的骑手
    start = free_time
    time_used = time * 1.1
    finish = start + time_used

    task.append((id_order, id_rider, start, finish))
    heapq.heappush(heap, (finish, id_rider))

    # 打印分配结果
    print(f"[{id_order}] 分配给骑手 {id_rider}, 预计 {time} 分钟")

    # 打印骑手状态
    riders = {1: "空闲", 2: "空闲", 3: "空闲"}
    for _, r, _, f in task:
        if f > start:  # 当前时刻还没送完
            riders[r] = "配送中"
    print(f"骑手状态: 1-{riders[1]}, 2-{riders[2]}, 3-{riders[3]}")

# 模拟骑手完成订单
for id_order, id_rider, start, finish in sorted(task, key=lambda x: x[3]):
    print(f"[骑手 {id_rider}] 完成订单 {id_order}, 实际耗时 {finish:.1f} 分钟")
    # 完成后的骑手状态
    riders = {1: "空闲", 2: "空闲", 3: "空闲"}
    for _, r, _, f in task:
        if f > finish:
            riders[r] = "配送中"
    print(f"骑手状态: 1-{riders[1]}, 2-{riders[2]}, 3-{riders[3]}")


[A100] 分配给骑手 1, 预计 30 分钟
骑手状态: 1-配送中, 2-空闲, 3-空闲
[A101] 分配给骑手 2, 预计 15 分钟
骑手状态: 1-配送中, 2-配送中, 3-空闲
[A102] 分配给骑手 3, 预计 25 分钟
骑手状态: 1-配送中, 2-配送中, 3-配送中
[A103] 分配给骑手 2, 预计 20 分钟
骑手状态: 1-配送中, 2-配送中, 3-配送中
[骑手 2] 完成订单 A101, 实际耗时 16.5 分钟
骑手状态: 1-配送中, 2-配送中, 3-配送中
[骑手 3] 完成订单 A102, 实际耗时 27.5 分钟
骑手状态: 1-配送中, 2-配送中, 3-空闲
[骑手 1] 完成订单 A100, 实际耗时 33.0 分钟
骑手状态: 1-空闲, 2-配送中, 3-空闲
[骑手 2] 完成订单 A103, 实际耗时 38.5 分钟
骑手状态: 1-空闲, 2-空闲, 3-空闲


In [4]:
# Python3 —— 最小改动版本（保留原变量名）
import sys
import math
import heapq

heap = [(0,1),(0,2),(0,3)]      # (空闲时间, 骑手id)
task = []                        # (id_order, id_rider, start, finish)
finish_list = []                 # (finish, id_rider, id_order, start)

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue
    if line.lower() == "end":
        break

    # ---- 解析输入：订单号-地址-预计时间(分钟) ----
    id_order, rest = line.split("-", 1)
    # 地址可能含“-”，因此用 rsplit 只从右侧切一次时间
    address, time = rest.rsplit("-", 1)
    time = int(time)

    # ---- 取最早空闲的骑手，计算开始/结束时间 ----
    t, id_rider = heapq.heappop(heap)  # t 即该骑手当前空闲时间
    start = t
    time_used = math.ceil(time * 1.1)  # 实际耗时=ceil(预计*1.1)
    finish = start + time_used

    task.append((id_order, id_rider, start, finish))
    finish_list.append((finish, id_rider, id_order, start))
    heapq.heappush(heap, (finish, id_rider))

    # ---- 接单时输出：分配信息 + “当前时刻=start”的骑手状态 ----
    print(f"[{id_order}] 分配给骑手 {id_rider}, 预计 {time} 分钟")

    state = {1: "空闲", 2: "空闲", 3: "空闲"}
    # 在当前时刻 start，凡 finish > start 的骑手都处于“配送中”
    for _, r, s, f in task:
        if f > start:
            state[r] = "配送中"
    print(f"骑手状态: 1-{state[1]}, 2-{state[2]}, 3-{state[3]}")

# ---- 所有接单后：按完成时间升序输出完成信息，并给出完成时刻的骑手状态 ----
finish_list.sort()  # 根据 finish 升序
for finish_1, id_rider, id_order in [(x[0], x[1], x[2]) for x in finish_list]:
    # 查该单的 start（用于打印“实际耗时”）
    # 注：保持最小改动，不额外建索引，直接在 task 中查询
    for oid, rid, start, fin in task:
        if oid == id_order and rid == id_rider and fin == finish_1:
            real_used = fin - start
            break
    print(f"[骑手 {id_rider}] 完成订单 {id_order}! 实际耗时 {real_used} 分钟")

    # 完成时刻 finish_1 的骑手状态
    state = {1: "空闲", 2: "空闲", 3: "空闲"}
    for _, r, s, f in task:
        if f > finish_1:
            state[r] = "配送中"
    print(f"骑手状态: 1-{state[1]}, 2-{state[2]}, 3-{state[3]}")


[A100] 分配给骑手 1, 预计 30 分钟
骑手状态: 1-配送中, 2-空闲, 3-空闲
[A101] 分配给骑手 2, 预计 15 分钟
骑手状态: 1-配送中, 2-配送中, 3-空闲
[A102] 分配给骑手 3, 预计 25 分钟
骑手状态: 1-配送中, 2-配送中, 3-配送中
[A103] 分配给骑手 2, 预计 20 分钟
骑手状态: 1-配送中, 2-配送中, 3-配送中
[骑手 2] 完成订单 A101! 实际耗时 17 分钟
骑手状态: 1-配送中, 2-配送中, 3-配送中
[骑手 3] 完成订单 A102! 实际耗时 28 分钟
骑手状态: 1-配送中, 2-配送中, 3-空闲
[骑手 1] 完成订单 A100! 实际耗时 33 分钟
骑手状态: 1-空闲, 2-配送中, 3-空闲
[骑手 2] 完成订单 A103! 实际耗时 22 分钟
骑手状态: 1-空闲, 2-空闲, 3-空闲


In [6]:
# Python3  —— 保留 state/list 的写法
import sys
import math
import heapq

heap = [(0,1),(0,2),(0,3)]                 # (空闲时间, 骑手id)
task = []                                   # (id_order, id_rider, start, finish)
finish_list = []                            # (finish, id_rider, id_order, start)

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue
    if line.lower() == "end":
        break

    # ---- 解析（地址可能包含 '-'，用 rsplit 取时间）----
    id_order, rest = line.split("-", 1)
    address, time = rest.rsplit("-", 1)     # 关键改动：rsplit
    time = int(time)

    # ---- 分配给最早空闲的骑手 ----
    t, id_rider = heapq.heappop(heap)       # t 即“当前分配时刻”
    start = t
    time_used = math.ceil(time * 1.1)       # 题意一般是 ×1.1 向上取整；若要小数改成 time*1.1
    finish = start + time_used

    task.append((id_order, id_rider, start, finish))
    finish_list.append((finish, id_rider, id_order, start))
    heapq.heappush(heap, (finish, id_rider))

    # ---- 使用 state / state_rider 输出“此刻”的状态（此刻 = start）----
    state = ["空闲","配送中"]               # 按你的写法保留
    state_rider = {1: state[0], 2: state[0], 3: state[0]}
    for _, r, s, f in task:
        if f > start:                        # 关键改动：与“当前时刻”比较
            state_rider[r] = state[1]

    print(f"[{id_order}] 分配给骑手 {id_rider}, 预计 {time} 分钟")
    print(f"骑手状态: 1-{state_rider[1]}, 2-{state_rider[2]}, 3-{state_rider[3]}")

# ---- 完成阶段：按完成时间输出，并用同一套 state 显示完成时刻的状态 ----
finish_list.sort()
for finish_1, id_rider, id_order, start in finish_list:
    real_used = finish_1 - start
    print(f"[骑手 {id_rider}] 完成订单 {id_order}! 实际耗时 {real_used} 分钟")

    state = ["空闲","配送中"]
    state_rider = {1: state[0], 2: state[0], 3: state[0]}
    for _, r, s, f in task:
        if f > finish_1:
            state_rider[r] = state[1]
    print(f"骑手状态: 1-{state_rider[1]}, 2-{state_rider[2]}, 3-{state_rider[3]}")


[A100] 分配给骑手 1, 预计 30 分钟
骑手状态: 1-配送中, 2-空闲, 3-空闲
[A101] 分配给骑手 2, 预计 15 分钟
骑手状态: 1-配送中, 2-配送中, 3-空闲
[A102] 分配给骑手 3, 预计 25 分钟
骑手状态: 1-配送中, 2-配送中, 3-配送中
[A103] 分配给骑手 2, 预计 20 分钟
骑手状态: 1-配送中, 2-配送中, 3-配送中
[骑手 2] 完成订单 A101! 实际耗时 17 分钟
骑手状态: 1-配送中, 2-配送中, 3-配送中
[骑手 3] 完成订单 A102! 实际耗时 28 分钟
骑手状态: 1-配送中, 2-配送中, 3-空闲
[骑手 1] 完成订单 A100! 实际耗时 33 分钟
骑手状态: 1-空闲, 2-配送中, 3-空闲
[骑手 2] 完成订单 A103! 实际耗时 22 分钟
骑手状态: 1-空闲, 2-空闲, 3-空闲
